In [3]:
import numpy as np
# np.save('Nv.npy', Nv)
Nv = np.load('Nv.npy')
print(Nv)
Sv = np.load('Sv.npy')
print(Sv)

[0.000e+00 1.000e-02 2.000e-02 ... 6.738e+01 6.739e+01 6.740e+01]
[1.         1.         1.         ... 0.32904425 0.32245421 0.31593402]


In [6]:
import math
import numpy as np
from scipy.integrate import solve_ivp
import os
os.environ['KMP_DUPLICATE_LIB_OK']= 'TRUE'
from scipy import interpolate
param = (interpolate.InterpolatedUnivariateSpline(Nv, Sv),)   

def solve(N_span,ini_state):
    def tensor(t, state, spline):
        z, x, y = state
        sigma = float(spline(t))
        dz = 1.5 * sigma - 1
        dx = -3 * x + 1.5 * sigma * x - math.exp(z) * y
        dy = -y + 1.5 * sigma * y + math.exp(z) * x
        return [dz, dx, dy]
    def subhorizon(N, state, spline):
        return state[0]-5
    subhorizon.terminal = True; subhorizon.direction = 1
    def jacobian(N, state, spline):
        z, x, y = state
        sigma = float(spline(N))
        jac = np.array(
            (
                (0, 0, 0),
                (-math.exp(z)*y, -3+1.5*sigma, -math.exp(z)),
                (math.exp(z)*x, math.exp(z), -1+1.5*sigma),
            ),
            dtype = np.float64,
        )
        return jac
    
    result = solve_ivp(tensor, N_span, ini_state, 
                        method='LSODA', 
                        #t_eval=N,
                        dense_output=True,
                        events=[subhorizon,],
                        args=param,
                        rtol=1e-6, atol=[1e-10, 1e-20, 1e-20],
                        jac=jacobian,
                        )
    return result.sol

In [21]:
from bson import ObjectId
import pymongo
from tqdm import tqdm
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["solve"]
collection = db["ivp"]


for document in tqdm(collection.find({
    'sol':{"$exists":False}
})): 
    sol = solve(N_span=document['N_span'],ini_state=document['ini_state'])
    N_effective = Nv[(Nv >= sol.t_min) & (Nv <= sol.t_max)]
    [zf_this, xf_this, yf_this] = sol(N_effective)
    collection.update_one(
        {'_id': ObjectId(document['_id'])},  
        {'$set': {
            't_min': sol.t_min,
            't_max': sol.t_max,
            'zf_this': zf_this.tolist(),
            'xf_this': xf_this.tolist(),
            'tf_this': yf_this.tolist(),
            }}  
    )



0it [00:00, ?it/s]

10000it [24:38,  6.76it/s]
